<a href="https://colab.research.google.com/github/SurapaneniJayasri/FMML_LAB1/blob/main/Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [2]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-2-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [3]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [4]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [5]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [6]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [7]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [8]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [9]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [10]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [11]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [12]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

# Answer for first question

increasing the Percentage of Validation Set:

Positive Impact on Model Performance: When you increase the percentage of data allocated to the validation set, you are effectively reducing the amount of training data. This can lead to several outcomes: Better Generalization: A larger validation set can help you better estimate the model's performance on unseen data. It can reveal issues with overfitting because the model has less data to overfit on during training. Robustness Testing: A larger validation set allows for more robust testing of the model's performance, ensuring it performs well on a broader range of data. Negative Impact on Model Performance: However, there are trade-offs to consider: Reduced Training Data: With more data allocated to validation, you have less data available for training. This can result in a model that doesn't learn as effectively, potentially leading to underfitting. Slower Training: Training may take longer as the model has fewer samples for learning. Finding the Right Balance: The optimal size for a validation set depends on the dataset, the complexity of the model, and the amount of data available. Cross-validation techniques can help mitigate the negative impact of a smaller training set while providing robust validation. Reducing the Percentage of Validation Set:
Positive Impact on Model Performance: Reducing the percentage of data allocated to the validation set means you have more data available for training. This can lead to the following outcomes:

2.Answer For Second Question

Reduced Training Data: With more data allocated to validation, you have less data available for training. This can result in a model that doesn't learn as effectively, potentially leading to underfitting. Slower Training: Training may take longer as the model has fewer samples for learning. Finding the Right Balance: The optimal size for a validation set depends on the dataset, the complexity of the model, and the amount of data available. Cross-validation techniques can help mitigate the negative impact of a smaller training set while providing robust validation. Reducing the Percentage of Validation Set:

Positive Impact on Model Performance: Reducing the percentage of data allocated to the validation set means you have more data available for training. This can lead to the following outcomes: Larger Validation Set:

A larger validation set can provide a more accurate estimate of how well the model  generalizes to unseen data. With a larger validation set, the model's performance on this set is likely to be a better indicator of its performance on the test set. However, if you allocate too much data to the validation set, you may reduce the amount of data available for training, potentially leading to underfitting. The model may not learn the underlying patterns effectively.

3.Answer For Third Question

Rule of Thumb: A common practice is to split your data into three sets: training, validation, and test. Common split ratios are 70-15-15, 80-10-10, or 60-20-20, where the first number represents the percentage allocated to training, the second to validation, and the third to testing. These ratios ensure a reasonable balance between training and validation, while still reserving a separate test set for final evaluation.

Size of Dataset: If you have a small dataset, you might lean towards a larger validation set because you have limited data for training. For very large datasets, a smaller validation set may be sufficient.

Complexity of Model: If you're working with a complex model (e.g., deep neural
networks), you may need a larger validation set to detect overfitting. Simpler models may require a smaller validation set.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [13]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [14]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


# Answer For First Question

Reducing Variance: Averaging validation accuracy over multiple splits helps reduce the impact of random variations in the data split. When you perform a single random split of your data into a training and validation set, the performance metric might be influenced by which data points ended up in the validation set. By repeatedly splitting the data into different subsets, you get a more stable estimate of the model's performance.

Better Generalization Estimate: Cross-validation provides a better estimate of how well your model will generalize to unseen data. It assesses how your model performs across different subsets of the data, which can be particularly useful when you have a limited dataset.

More Robust Model Selection: If you're using cross-validation for hyperparameter tuning or model selection, averaging the results over multiple folds helps you make more informed decisions about which model or set of hyperparameters performs best on average.



#Answer For Second Question

Averaging the validation accuracy across multiple splits using techniques like k-fold cross-validation does not directly provide a more accurate estimate of the test accuracy. Instead, it provides a more accurate estimate of how well your model is likely to perform on unseen data, which is similar to what the test accuracy measures. Here's how it works:

Validation Accuracy: When you use cross-validation, you split your dataset into multiple folds (e.g., 5 or 10) and train and validate your model on different combinations of these folds. You calculate the validation accuracy for each fold, and then you typically average these validation accuracies to obtain a more stable estimate of how well your model performs on data it hasn't seen during training.

Test Accuracy: The test accuracy is typically calculated on a separate, held-out test dataset that the model has never seen during training or cross-validation. It provides an estimate of how well the model generalizes to completely new, unseen data.

While cross-validation does provide a more robust estimate of how well the model will perform on unseen data compared to a single validation split, it's important to note that the test accuracy is still the most direct and accurate estimate of a model's performance on truly unseen data. The test dataset is entirely separate from the training and validation data and serves as an unbiased evaluation of the model's generalization ability.



# Answer For Third Question

The number of iterations or epochs in machine learning, particularly in training deep learning models, can have a significant impact on the estimate of the model's performance. In general, increasing the number of iterations can lead to a better estimate, but it's essential to strike a balance to avoid overfitting and excessive computational cost. Here's how it works:

Effect of Number of Iterations:

Early Iterations: In the early iterations of training, the model is learning the underlying patterns in the training data. The training loss decreases, and the model becomes better at fitting the training data. Later Iterations: As you continue training for more iterations, the model may start to overfit the training data. It becomes increasingly specialized in capturing noise in the data, which can lead to a decrease in performance on unseen data (validation or test sets). Finding the Right Number of Iterations:

Underfitting: If you stop training too early (few iterations), the model may underfit the data, resulting in suboptimal performance. Overfitting: If you train for too many iterations, the model may overfit the training data and not generalize well to new data. Early Stopping: To find the right balance, many practitioners use techniques like early stopping. This involves monitoring the model's performance on a validation set during training and stopping when performance starts to degrade (indicating overfitting). Regularization Techniques: Regularization techniques like dropout, weight decay, and batch normalization can help mitigate the overfitting risk associated with training for a large number of iterations.



# Answer For Fourth Question

Increasing the number of iterations can help to some extent when dealing with a small training dataset or a small validation dataset, but it has its limitations. Here's how increasing iterations can impact the situation:

Small Training Dataset:

If you have a very small training dataset, increasing the number of iterations can allow the model to see and learn from the limited data more thoroughly. It might help the model capture more patterns and improve its performance. However, there's a risk of overfitting when using a small training dataset, especially if you train for a large number of iterations. The model may start to memorize the training data rather than generalize from it. In such cases, regularization techniques like dropout, weight decay, or using a simpler model architecture may be necessary to prevent overfitting. Small Validation Dataset:

A small validation dataset can make it challenging to reliably estimate the model's performance. Increasing the number of iterations can help in the sense that it allows the model to be evaluated more times on the validation set, potentially leading to a more stable estimate of performance. However, the fundamental issue of a small validation dataset remains. It may not adequately represent the diversity of data the model will encounter in real-world scenarios. This can limit the model's ability to generalize effectively.